In [1]:
import csv
import sys, os
from scipy.sparse import *
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import *
from sklearn.linear_model import *
import cPickle as pkl
from sklearn.svm import *
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm
from sklearn.preprocessing import *

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# print "loading data"
# with open('X.pkl','rb') as f:
#     training_data = pkl.load(f)
print "loading labels"
with open('Y.pkl','rb') as f:
    training_labels = pkl.load(f)

loading labels


In [4]:
def train(X, y, clf):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    print "fitting"
    clf.fit(X_train, y_train)

    print "predicting"
    preds = clf.predict(X_val)

    from sklearn.metrics import mean_absolute_error
    print("RF MSE: %.4f" % mean_absolute_error(y_val, preds))

In [6]:
clf = RandomForestRegressor(n_estimators=32, n_jobs=-1)

v = DictVectorizer()
n_train = 40000
td = v.fit_transform(training_data[:n_train])
ty = training_labels[:n_train]
train(td, ty, clf)

fitting
predicting
RF MSE: 169.5150


In [7]:
valx = v.transform(training_data[-10000:])
valy = training_labels[-10000:]
preds = clf.predict(valx)
from sklearn.metrics import mean_absolute_error
print("RF MSE: %.4f" % mean_absolute_error(valy, preds))

RF MSE: 166.0119


In [16]:
print training_data[6]

{'num_plays': 7288904, 'num_listeners': 21051, 'average_listener_age': 22.985186446449855, 'user_play_count': 1904, 'm': 1, 'user_avg_popularity': 64.4, 'user_region_Europe': 1, 'share_genre': 1, 'user_age': 25.0, 'artist_region_Europe': 1, 'prop_male_listeners': 0.6824414202394352, 'share_sex': 1, 'share_region': 1, 'num_user_artists': 20, 'artist_popularity': 75}


In [5]:
with open('artist_lookup', 'rb') as f:
    artist_lookup = pkl.load(f)

In [7]:
with open('user_lookup.pkl', 'rb') as f:
    user_lookup = pkl.load(f)

In [27]:
print user_lookup['fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d']
print artist_lookup[artist_lookup.keys()[6]]

{'region': 'Europe', 'average_popularity': 56.85, 'sex': 'f', 'favorite_genres': Counter({u'neo mellow': 7, u'rock': 7, u'classic rock': 5, u'alternative rock': 5, u'pop rock': 5, u'pop christmas': 4, u'permanent wave': 3, u'folk-pop': 3, u'pop': 3, u'singer-songwriter': 3, u'folk christmas': 3, u'post-teen pop': 3, u'dance pop': 3, u'mellow gold': 2, u'funk rock': 2, u'swedish indie pop': 2, u'indie rock': 2, u'hollywood': 2, u'album rock': 2, u'viral pop': 2, u'indie folk': 2, u'indie pop': 2, u'chamber pop': 2, u'indie christmas': 2, u'neo-psychedelic': 2, u'stomp and holler': 2, u'post-grunge': 2, u'psychedelic rock': 2, u'folk rock': 1, u'acoustic pop': 1, u'protopunk': 1, u'classic swedish pop': 1, u'merseybeat': 1, u'british invasion': 1, u'soul': 1, u'new wave': 1, u'europop': 1, u'broadway': 1, u'hard rock': 1, u'electronic': 1, u'funk metal': 1, u'swedish folk pop': 1, u'britpop': 1, u'motown': 1, u'deep swedish indie pop': 1, u'brill building pop': 1, u'glam rock': 1, u'movi

In [8]:
train_data = {}
with open('data/train.csv', 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = row[2]
    
        if not user in train_data:
            train_data[user] = {}
        
        train_data[user][artist] = int(plays)

# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [43]:
median_data = []
with open('data/train.csv', 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    count = 0
    next(train_csv, None)
    for row in train_csv:
        median_data.append({})
        user   = row[0]
        artist = row[1]
        median_data[count]['user_id'] = user
        median_data[count]['artist_id'] = artist
        median_data[count]['user_median'] = user_medians[user]
        median_data[count]['user_age'] = user_lookup[user]['age']
        median_data[count]['region_share'] = artist_lookup[artist][user_lookup[user]['region']]
        median_data[count]['total_plays'] = user_lookup[user]['total_plays']
        median_data[count]['age_diff'] = abs(user_lookup[user]['age'] - artist_lookup[artist]['average_age'])
        median_data[count]['populatiry'] = artist_lookup[artist]['popularity']
        male = (user_lookup[user]['sex'] == 'm')
        female = (user_lookup[user]['sex'] == 'f')
        m_interaction = 0
        f_interaction = 0
        if male:
            m_interaction = artist_lookup[artist]['prop_male']
        if female:
            f_interaction = artist_lookup[artist]['prop_female']
        median_data[count]['m_interaction'] = m_interaction
        median_data[count]['f_interaction'] = f_interaction
        count += 1

In [ ]:
clf = RandomForestRegressor(n_estimators=40, n_jobs=-1)

v = DictVectorizer()
n_train = 20000
td = maxabs_scale(v.fit_transform(median_data[:n_train]))
ty = training_labels[:n_train]
train(td, ty, clf)

fitting


In [47]:
valx = maxabs_scale(v.transform(median_data[-10000:]))
valy = training_labels[-10000:]
preds = clf.predict(valx)
from sklearn.metrics import mean_absolute_error
print("RF MSE: %.4f" % mean_absolute_error(valy, preds))

RF MSE: 133.1980


In [49]:
soln_data = []
ids = []
with open('data/test.csv', 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    count = 0
    next(train_csv, None)
    for row in train_csv:
        soln_data.append({})
        id = row[0]
        user   = row[1]
        artist = row[2]
        ids.append(id)
        soln_data[count]['user_id'] = user
        soln_data[count]['artist_id'] = artist
        soln_data[count]['user_median'] = user_medians[user]
        soln_data[count]['user_age'] = user_lookup[user]['age']
        soln_data[count]['region_share'] = artist_lookup[artist][user_lookup[user]['region']]
        soln_data[count]['total_plays'] = user_lookup[user]['total_plays']
        soln_data[count]['age_diff'] = abs(user_lookup[user]['age'] - artist_lookup[artist]['average_age'])
        male = (user_lookup[user]['sex'] == 'm')
        female = (user_lookup[user]['sex'] == 'f')
        m_interaction = 0
        f_interaction = 0
        if male:
            m_interaction = artist_lookup[artist]['prop_male']
        if female:
            f_interaction = artist_lookup[artist]['prop_female']
        soln_data[count]['m_interaction'] = m_interaction
        soln_data[count]['f_interaction'] = f_interaction
        count += 1

In [50]:
test_x = v.transform(soln_data)
test_y = clf.predict(test_x)

In [54]:
print len(ids)
print len(test_y)

4154804
4154804


In [55]:
with open('solution.csv', 'w') as soln_fh:
    soln_csv = csv.writer(soln_fh,
                          delimiter=',',
                          quotechar='"',
                          quoting=csv.QUOTE_MINIMAL)
    soln_csv.writerow(['Id', 'plays'])
    for id,y in zip(ids,test_y):
        soln_csv.writerow([id, y])